## Download the images


We can use **GoogleDriveDownloader** form **google_drive_downloader** library in Python to download the shared files from the shared Google drive link: https://drive.google.com/file/d/1f7uslI-ZHidriQFZR966_aILjlkgDN76/view?usp=sharing

The file id in the above link is: **1f7uslI-ZHidriQFZR966_aILjlkgDN76**

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1f7uslI-ZHidriQFZR966_aILjlkgDN76',
                                    dest_path='content/eye_gender_data.zip',
                                    unzip=True)

Unzipping...Done.


We have all the files from the shared Google drive link downloaded in the colab environment.

## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [1]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set
from sklearn.metrics import f1_score

## Loading and preparing training data


In [6]:
labels = pd.read_csv("content/eye_gender_data/Training_set.csv")   # loading the labels
file_paths = [[fname, 'content/eye_gender_data/train/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')

X_train = [] # initialize an empty numpy array
y_train = []
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
    img_array = cv2.imread(train_data['filepaths'][i])   # converting the image to gray scale
    new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
    X_train.append(new_img_array)
    y_train.append(train_data['label'][i])

In [7]:
X_train=np.array(X_train)
y_train=np.array(y_train)

# labeling male as 1 and female as 0
y_train=np.where(y_train=="male",1,0)

In [8]:
print(X_train.shape)
print(y_train.shape)

(9220, 100, 100, 3)
(9220,)


## Data Pre-processing
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [9]:
# reshape for feeding into the model
X_train = X_train.reshape(X_train.shape[0], 100, 100, 3)

print('\nX_train.shape: {}, of {}'.format(X_train.shape, X_train.dtype))


X_train.shape: (9220, 100, 100, 3), of uint8


## Building Model & Hyperparameter tuning
Now we are finally ready, and we can train the model.


In [10]:
# define input shape
INPUT_SHAPE = (100, 100, 3)

# get the VGG19 model
vgg_layers = tf.keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, 
                                               input_shape=INPUT_SHAPE)

vgg_layers.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [11]:
# Fine-tune all the layers
for layer in vgg_layers.layers:
    layer.trainable = True

# Check the trainable status of the individual layers
for layer in vgg_layers.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x0000027A9B598CD0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000027A9B5FCAC0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000027AA7108610> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000027AA7108280> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000027A9B5DEA00> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000027A9B5E26D0> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000027A9B5B49A0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000027A9B5DE4C0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000027A9B5EAA60> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000027A9B5ECD60> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000027A9B7331C0> True
<tensorflow.python.keras.layer

In [12]:
# define sequential model
model = tf.keras.models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_layers)

# add flatten layer
model.add(tf.keras.layers.Flatten())

# add dense layers with some dropout
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.3))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.3))

# add output layer
model.add(tf.keras.layers.Dense(2, activation='softmax'))

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# view model layers
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 3, 3, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1179904   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 5

## Validate the model


In [ ]:
EPOCHS = 100
X_train = X_train / 255.
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, 
                                               restore_best_weights=True,
                                               verbose=1)

history = model.fit(X_train, y_train,
                    batch_size=32,
                    callbacks=[es_callback], 
                    validation_split=0.1, epochs=EPOCHS,
                    verbose=1)

## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. Test.csv).

#### Load Test Set
Load the test data on which final submission is to be made.

In [14]:
labels = pd.read_csv("content/eye_gender_data/Testing_set.csv")   # loading the labels
file_paths = [[fname, 'content/eye_gender_data/test/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_data = pd.merge(images, labels, how = 'inner', on = 'filename')

X_test = [] # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(test_data)):
    img_array = cv2.imread(test_data['filepaths'][i])   # converting the image to gray scale
    new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
    X_test.append(new_img_array)

In [15]:
X_test=np.array(X_test)
X_test.shape

(2305, 100, 100, 3)

In [16]:
X_test = X_test / 255.

## Data Pre-processing on test_data


In [18]:
# reshape for feeding into the model
X_test = X_test.reshape(X_test.shape[0], 100, 100, 3)

print('\nX_test.shape: {}, of {}'.format(X_test.shape, X_test.dtype))


X_test.shape: (2305, 100, 100, 3), of float64


### Make Prediction on Test Dataset
Time to make a submission!!!

In [ ]:
predictions = model.predict(X_test)
predictions[:5]

In [ ]:
prediction_labels = np.argmax(predictions, axis=1)
prediction_labels[:10]

## **How to save prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **How to save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Well Done! 👍**
You are all set to make a submission. Let's head to the **[challenge page](https://dphi.tech/challenges/4-week-deep-learning-online-bootcamp-final-assignment-sex-determination-by-morphometry-of-eyes/144/submit)** to make the submission.